In [1]:
import numpy as np 
import pandas as pd

In [4]:
# add the path to the curration file temporarily to the sys path
import sys
sys.path
sys.path.append("C:/Users/Danial Nasir Awan'/Desktop/Notion/Projects and portfolios/Projects/prophet_present_project/liabraries")
import curration_test_3 as cu 

Cleaning and prepring l60 for testing

In [5]:
import pandas as pd 
testing = pd.read_csv('../Raw/mfnl6020241015_20241215.csv')
testing.shape

(24376, 73)

In [6]:
test = cu.curate_testing_data(testing,'daily')
test

,sku,purchase_date,sell_qty,business_order,per_unit_cost
0,PSD226C8TFCF-2,2024-10-20,1,0,67.66
1,PSD226C8TFCF-2,2024-10-22,1,0,67.66
2,PSD22FRHJFGH-4,2024-10-18,2,0,40.26
3,PSD22FRHJFGH-4,2024-12-10,1,0,36.19
4,PSD22J2RKFK8,2024-10-15,6,0,32.77
...,...,...,...,...,...
16872,PSDZZY6F3QVJ,2024-12-08,1,0,190.99
16873,PSDZZY6F3QVJ,2024-12-09,2,0,190.99
16874,PSDZZY6F3QVJ,2024-12-10,1,0,190.99
16875,PSDZZY6F3QVJ,2024-12-11,3,0,190.99


In [7]:
testing_data = test 
testing_data['purchase_date'] = pd.to_datetime(testing_data['purchase_date'])
testing_data[testing_data['purchase_date'].dt.date >= pd.Timestamp('2024-10-25').date()]
# df = pd.DataFrame(columns=['sku','purchase_date','sell_qty','bussiness_oder'])
tests = []
for sku,test in testing_data.groupby('sku'):
    # print(test)
    test['purchase_date'] = pd.to_datetime(test['purchase_date'])
    
    min_date = test['purchase_date'].min()
    max_date = test['purchase_date'].max()
    date_range = pd.date_range(start=min_date, end=max_date, freq='D')

    # Create a new DataFrame with the full date range
    date_df = pd.DataFrame({'purchase_date': date_range})

    # Merge with the original DataFrame to fill missing dates
    test = pd.merge(date_df, test, on='purchase_date', how='left')

    # Optionally, fill missing values (forward-fill, backfill, or set to default values)
    test['sku'] = test['sku'].ffill()  # Example: Forward-fill SKUs
    test['sell_qty'] = test['sell_qty'].fillna(0)  # Default missing sell_qty to 0
    test['business_order'] = test['business_order'].fillna(0)  # Default missing business_order to 0
    test['cap'] = test['sell_qty'].max()+1
    test['floor'] = 0
    tests.append(test)
df  = pd.concat(tests, ignore_index=True)
testing_data = df
print(testing_data.shape)
testing_data.head(2)

(77456, 7)


,purchase_date,sku,sell_qty,business_order,per_unit_cost,cap,floor
0,2024-10-20,PSD226C8TFCF-2,1.0,0.0,67.66,2.0,0
1,2024-10-21,PSD226C8TFCF-2,0.0,0.0,NaN,2.0,0


In [8]:
print(testing_data['purchase_date'].min(), testing_data['purchase_date'].max())

2024-10-15 00:00:00 2024-12-15 00:00:00


In [9]:
testing_data.to_csv('../currated/testing_data_daily_currated_20241015_20241215.csv')

currating training data daily


In [11]:
mfn =pd.read_csv('../raw/mfn.csv')
fba = pd.read_csv('../raw/fba.csv')
prod = pd.read_csv('../raw/prod.csv')

C:\Users\Danial Nasir Awan'\AppData\Local\Temp\ipykernel_11144\1468330613.py:3: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  prod = pd.read_csv('../raw/prod.csv')


In [ ]:
# # add the path to the curration file temporarily to the sys path
# import sys
# sys.path
# sys.path.append("C:/Users/Danial Nasir Awan'/Desktop/Notion/Projects and portfolios/Projects/prophet_amazon_02/liabraries")
# import curration_test_3 as cu 

In [12]:
product_data, mfn_data, fba_sales, _, df= cu.curate_table(prod,mfn,fba,'daily')

In [15]:
df 

,sku,purchase_date,sell_qty,business_order
0,PSD22J2RKFK8,2022-09-13 09:15:49,1,1
1,PSD22J2RKFK8,2022-09-16 11:54:40,2,0
2,PSD22J2RKFK8,2022-09-16 13:57:22,2,0
3,PSD22J2RKFK8,2022-09-19 10:30:46,1,0
4,PSD22J2RKFK8,2022-09-20 13:21:04,1,1
...,...,...,...,...
228542,PSDZZY6F3QVJ,2024-06-03 20:21:05,1,1
228543,PSDZZY6F3QVJ,2024-06-06 18:53:16,1,1
228544,PSDZZY6F3QVJ,2024-06-22 22:06:36,1,1
228545,PSDZZY6F3QVJ,2024-07-23 15:29:00,1,1


In [16]:
print(df['purchase_date'].min(), df['purchase_date'].max())

2021-10-01 10:24:32 2024-10-25 17:13:41


In [17]:
df.to_csv('../currated/train_data_daily_20211001_20241025.csv')

getting top skus which have a lot of sales

In [20]:
# training_sku_df = pd.read_csv('../currated/train_data_daily_20211001_20241025.csv')
# testing_top_sku_df = pd.read_csv('../currated/testing_data_daily_currated_20241015_20241215.csv')
# top_skus = pd.read_csv('../currated/top150skus2024.csv')
# training_sku_df = training_sku_df[training_sku_df['sku'].isin(top_skus['sku'])]
# testing_top_sku_df = testing_top_sku_df[testing_top_sku_df['sku'].isin(top_skus['sku'])]

In [ ]:
# training_sku_df.head(2)

,Unnamed: 0,sku,purchase_date,sell_qty,business_order
0,0,PSD22J2RKFK8,2022-09-13 09:15:49,1,1
1,1,PSD22J2RKFK8,2022-09-16 11:54:40,2,0


In [ ]:
# testing_top_sku_df.head(2)

,Unnamed: 0,purchase_date,sku,sell_qty,business_order,per_unit_cost,cap,floor
118,118,2024-10-17,PSD23TWHT52X,1.0,0.0,156.62,3.0,0
119,119,2024-10-18,PSD23TWHT52X,0.0,0.0,NaN,3.0,0


In [ ]:
# top_sellers_training = training_sku_df.groupby('sku').agg({'sell_qty': sum, 'sku': 'size'}).rename(columns={'sku': 'RowCount'}).sort_values(by=['sell_qty', 'RowCount'], ascending=False).reset_index()
# top_sellers_training.head(4)

C:\Users\Danial Nasir Awan'\AppData\Local\Temp\ipykernel_17316\3477599331.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  top_sellers_training = training_sku_df.groupby('sku').agg({'sell_qty': sum, 'sku': 'size'}).rename(columns={'sku': 'RowCount'}).sort_values(by=['sell_qty', 'RowCount'], ascending=False).reset_index()


,sku,sell_qty,RowCount
0,PSDQBLCVLWTW,11537,8427
1,PSD8CHBKL6MJ,4142,2881
2,PSDYT65JHBNK,3632,2685
3,PSD6NVKC69NJ,2467,932


In [ ]:
# top_sellers_test = testing_top_sku_df.groupby('sku').agg({'sell_qty': sum, 'sku': 'size'}).rename(columns={'sku': 'RowCount'}).sort_values(by=['sell_qty', 'RowCount'], ascending=False).reset_index()
# top_sellers_test.head(4)

C:\Users\Danial Nasir Awan'\AppData\Local\Temp\ipykernel_17316\3846785291.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  top_sellers_test = testing_top_sku_df.groupby('sku').agg({'sell_qty': sum, 'sku': 'size'}).rename(columns={'sku': 'RowCount'}).sort_values(by=['sell_qty', 'RowCount'], ascending=False).reset_index()


,sku,sell_qty,RowCount
0,PSDYT65JHBNK,469.0,60
1,PSD6B28VJ233,440.0,55
2,PSDWXZTK64BB,411.0,62
3,PSDVY98HJBVN,386.0,60


In [ ]:
# # Find SKUs that are common in both DataFrames
# common_skus = pd.merge(top_sellers_test, top_sellers_training, on='sku', suffixes=('_test', '_train'))

# # Sum the sell_qty values from both DataFrames (optional, depending on your need)
# common_skus['total_sell_qty'] = common_skus['sell_qty_test'] + common_skus['sell_qty_train']

# # Sort by the total_sell_qty in descending order
# sorted_common_skus = common_skus.sort_values(by='total_sell_qty', ascending=False)

# # Select the top 20 SKUs
# top_20_common_skus = sorted_common_skus.head(20)

In [ ]:
# top_20_common_skus.to_csv('../currated/top_skus_with_sell_qty.csv')